In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

from tensorflow.keras import Sequential,layers,losses,utils,Input
from tensorflow.keras.layers import Dense,LSTM,Dropout,concatenate,Flatten, Conv1D, MaxPooling1D,Activation,RepeatVector,TimeDistributed
import tensorflow as tf
# from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional,GRU,Lambda,Dot,Concatenate


import matplotlib.pyplot as plt
import matplotlib

from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error
from sklearn.decomposition import PCA


from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras import optimizers

import seaborn as sns

In [45]:
data=pd.read_csv('hunhe.csv')

In [ ]:
data

In [ ]:
data.columns

In [48]:
col=[ 'year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2',
       'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN',  'WSPM','wd','shunyiPM2.5']
datacol=data[col]

In [ ]:
datacol.isnull().sum()

In [50]:
# 直接删除缺失值
datashan=datacol.dropna()

In [ ]:
datashan.isnull().sum()

In [ ]:
datashan.shape

In [ ]:
datashan.describe()

In [ ]:
# 拼接时间为一列，绘制一年时间降水分布图
# datag['时间(年月日时)']=datag['年(年)'].map(str)+''+datag['月(月)'].map(str)+''+datag['日(日)'].map(str)+''+datag['时(时)'].map(str)
datashan['datatime']=datashan['year'].map(str)+'-'+datashan['month'].map(str)+'-'+datashan['day'].map(str)+'-'+datashan['hour'].map(str)

In [ ]:
datashan.head(8)

In [ ]:
# datag=data.copy()
datashan['datatime']=pd.to_datetime(datashan['datatime'],format='%Y-%m-%d-%H',errors='coerce')
datashan.index=datashan['datatime']

In [ ]:
datashan.isnull().sum()

In [ ]:
# 给风向编码
datashan['wd'].unique()

In [ ]:
scale_mapper={'E':1, 'ENE':2, 'NNE':3, 'N':4, 'NNW':5, 'NW':6, 'NE':7, 'SW':8, 'SSW':9, 'WSW':10,
       'ESE':11, 'SE':12, 'S':13, 'WNW':14, 'SSE':15, 'W':16}
datashan['wd']=datashan['wd'].replace(scale_mapper)

In [60]:
chongcol=[ 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM','wd','shunyiPM2.5']
datag=datashan[chongcol]

In [61]:
datal=datashan[chongcol]

In [ ]:
datal

In [ ]:
# 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)

In [ ]:
datal.max()

In [ ]:
datag

In [ ]:
# 归一化
scaler = MinMaxScaler(feature_range=(0, 1))  # minmaxscaler 方法，用来做归一化
sel_col = datag.columns
# print(sel_col)
for col in sel_col:
    datag[col] = scaler.fit_transform(datag[col].values.reshape(-1, 1))  # reshape成为一列
print("Load dataset LEN: ", datag.shape[0])
print(sel_col)


In [ ]:
# 归一化后的数据
datag

In [ ]:
rc = {'font.sans-serif': ['FangSong']}
sns.set(context='notebook', style='ticks', rc=rc,font_scale=1.5)
# sns.set(style="white",font_scale=1.5)#设置主题，文本大小
g=sns.jointplot(data=datag, x="shunyiPM2.5", y="PM2.5", kind="reg",color='#098154',marker='^')
g.fig.set_size_inches(10,8)
plt.savefig('pm2.5-shunyipm2.5.jpg')

In [ ]:
datag.corr()

In [ ]:
# 绘制相关性分析图
import seaborn as sns
# rc = {'font.sans-serif': ['FangSong']}
# sns.set(context='notebook', style='ticks', rc=rc,font_scale=1.5)
sns.clustermap(datag.corr(), xticklabels=True, yticklabels='auto',fmt='d', cmap='RdYlBu')
# plt.title('各指标相关性分析图')
sns.savefig('全特征热图.jpg')

In [ ]:
X = []
Y = []
columns_name = list(datag.columns)
y_index = columns_name.index('PM2.5')
tmp = np.array(datag, dtype=np.float32)
print("The shape of tmp is:")
print(tmp.shape)
pca = PCA(n_components=0.97)# 保证降维后的数据保持90%的信息
tmp2 = pca.fit_transform(tmp[:,1:])
print("The shape of tmp2 is:")
print(tmp2.shape)  # PCA降维y_index要改，或者先提取出来
print("input dim is:")
print(tmp2.shape[1])

In [39]:
seqX=120
seqY=1
day_delay=1
gap=1
X1=[]
for i in range(tmp2.shape[0] - (seqX+seqY+day_delay)*gap):
        # seqX, seqX+day_delay, seqX+day_delay+seqY
        # format: i+x*gap
        X.append(tmp2[i:(i + seqX*gap):gap, :])  # tmp2是PCA降维后的特征，如果是tmp那就是原始特征
        X1.append(tmp2[(i+seqX*gap): (i+(seqX+day_delay)*gap): gap,:])
#         Y.append(tmp[(i+(seqX+day_delay)*gap): (i+(seqX+day_delay+seqY)*gap): gap, y_index])
        Y.append(tmp[(i+seqX*gap): (i+(seqX+day_delay)*gap): gap, y_index])
X = np.array(X)
Y = np.array(Y)
X1=np.array(X1)

划分数据集

In [65]:
# x_train, x_test,  y_train, y_test = train_test_split(datax_trans, datay_trans, test_size = 0.2, random_state = 666,shuffle=False)

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
X1.shape

In [44]:
X = X.astype(np.float32)
Y = Y.astype(np.float32)
p = np.random.permutation(range(len(X)))
X,Y,X1 = X[p],Y[p],X1[p]

In [45]:
shendu_trainx, shendu_trainy = X[:int(0.9 * 1310)], Y[:int(0.9 * 1310)]  
jiqi_trainx, jiqi_trainy = X1[:int(0.9 * 1310)], Y[:int(0.9 * 1310)]  

In [46]:
shendu_testx, shendu_testy = X[int(0.9 * 1310):], Y[int(0.9 * 1310):] 
jiqi_testx, jiqi_testy = X1[int(0.9 * 1310):], Y[int(0.9 * 1310):] 

In [ ]:
shendu_trainx.shape

In [ ]:
jiqi_trainx.shape

In [ ]:
shendu_testx.shape

In [ ]:
jiqi_testx.shape

In [ ]:
jiqi_trainy.shape

In [ ]:
jiqi_testy.shape

转换成机器学习可以用的数据

In [54]:
realjiqi_trainx=jiqi_trainx.reshape(1179,8)

In [ ]:
realjiqi_trainx

In [58]:
realjiqi_testx=jiqi_testx.reshape(131,8)

In [ ]:
realjiqi_testx

In [ ]:
jiqi_testx

In [ ]:
realjiqi_testx.shape

保存数据

In [62]:
# 保存深度学习训练测试数据
# 将训练集存起来
nptrainx=np.reshape(shendu_trainx,(-1,8))
pdtrainx1=np.reshape(nptrainx,(-1,120,8))

In [ ]:
pdtrainx1.shape

In [64]:
pdshendutrainx=pd.DataFrame(nptrainx,columns=['t1','t2','t3','t4','t5','t6','t7','t8'])
pdshendutrainx.to_csv('shendux_train.csv',index=False)

In [65]:
nptestx=np.reshape(shendu_testx,(-1,8))

In [66]:
pdshendux_test=pd.DataFrame(nptestx,columns=['t1','t2','t3','t4','t5','t6','t7','t8'])
pdshendux_test.to_csv('shendux_test.csv',index=False)

In [67]:
pdshenduy_train=pd.DataFrame(shendu_trainy,columns=['y1'])
pdshenduy_train.to_csv('shenduy_train.csv',index=False)

In [68]:
pdshenduy_test=pd.DataFrame(shendu_testy,columns=['y1'])
pdshenduy_test.to_csv('shenduy_test.csv',index=False)

In [69]:
# 保存机器学习训练测试集
pdjiqitrainx=pd.DataFrame(realjiqi_trainx,columns=['t1','t2','t3','t4','t5','t6','t7','t8'])
pdjiqitrainx.to_csv('jiqix_train.csv',index=False)

In [70]:
pdjiqitestx=pd.DataFrame(realjiqi_testx,columns=['t1','t2','t3','t4','t5','t6','t7','t8'])
pdjiqitestx.to_csv('jiqix_test.csv',index=False)

In [71]:
pdjiqiy_train=pd.DataFrame(jiqi_trainy,columns=['y1'])
pdjiqiy_train.to_csv('jiqiy_train.csv',index=False)

In [72]:
pdjiqiy_test=pd.DataFrame(jiqi_testy,columns=['y1'])
pdjiqiy_test.to_csv('jiqiy_test.csv',index=False)

In [ ]:
jiqi_testy

In [ ]:
shendu_testy

读取深度学习数据

读取数据机器学习

In [21]:
# 读取训练集
dujiqix_train=pd.read_csv('jiqix_train.csv')
dujiqiy_train=pd.read_csv('jiqiy_train.csv')
# 读取测试集
dujiqix_test=pd.read_csv('jiqix_test.csv')
dujiqiy_test=pd.read_csv('jiqiy_test.csv')

In [ ]:
dujiqix_train

In [ ]:
dujiqix_test

In [ ]:
dujiqiy_train

方法一：线性预测

In [202]:
from sklearn.metrics import mean_absolute_error

In [89]:
from sklearn.linear_model import LinearRegression,RidgeCV,SGDRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score   #均方误差

In [344]:
# # # # 1.线性回归----此处使用基模型，参数均使用默认的，用于后面调参对比
# Lr=LinearRegression(normalize=False)
# Lr.fit(dujiqix_train,dujiqiy_train)
# xy_predict=Lr.predict(dujiqix_test)
# xy_modify = xy_predict*(xy_predict>=0)
# r2=r2_score(dujiqiy_test,xy_modify)
# # Mae=mean_squared_error(y_test,xy_modify)
# r2

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)

In [345]:
# xpreds = np.array(xy_modify)  # 转换为numpy形式
# xlabels = np.array(dujiqiy_test)
# xpreds= xpreds.reshape((-1,1))
# xfuture_len= xpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(xfuture_len):
#     xlabels[:, k] = xlabels[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
# for k in range(xfuture_len):
#     xpreds[:, k] = xpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [346]:
# np.sqrt(mean_squared_error(xlabels,xpreds))

In [347]:
# np.sqrt(mean_absolute_error(xlabels,xpreds))

In [343]:
# r2_score(xlabels,xpreds)

决策树

In [341]:
# tree_model=DecisionTreeRegressor()
# tree_model.fit(dux_train,duy_train)
# treey_pre=tree_model.predict(dux_test)
# ty_modify = treey_pre*(treey_pre>=0)
# jr=r2_score(duy_test,ty_modify)
# # treemae2=mean_squared_error(y_test,ty_modify2)
# # print(jr2,treemae2)
# jr

In [342]:
#  # 数据逆归一化
# maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
# print(maxmin)

In [349]:
# preds = np.array(ty_modify)  # 转换为numpy形式
# labels1 = np.array(duy_test)
# preds= preds.reshape((-1,1))
# future_len= preds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(future_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
# for k in range(future_len):
#     preds[:, k] = preds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
preds

In [ ]:
labels1

In [101]:
treemae=mean_squared_error(labels1,preds)

In [ ]:
treemae

In [ ]:
np.sqrt(mean_squared_error(labels1,preds))

In [ ]:
r2_score(labels1,preds)

随机森林

In [79]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rd=RandomForestRegressor(max_depth=11,max_features=6,min_samples_split=35,n_estimators=96,min_samples_leaf=9)     #基模型
rd.fit(dujiqix_train,dujiqiy_train)
suiy_pres=rd.predict(dujiqix_test)
yrand_modify = suiy_pres*(suiy_pres>=0)
suijir=r2_score(dujiqiy_test,yrand_modify)
suijir

In [ ]:
label = np.array(dujiqiy_test)
for k in range(1):
#     print(k)
    label[:, k] = label[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.array(dujiqiy_test)

In [114]:
spreds = np.array(yrand_modify)  # 转换为numpy形式
# labels1 = np.array(dujiqiy_test)
spreds= spreds.reshape((-1,1))
sfuture_len= spreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(sfuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(sfuture_len):
    spreds[:, k] = spreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
spreds

In [ ]:
np.sqrt(mean_squared_error(label,spreds))

In [ ]:
mean_absolute_error(label,spreds)

保存预测结果

In [208]:
# spreds1=spreds.reshape(-1,1)

In [118]:
spreds1=spreds.reshape(-1,1)
baocunspreds=pd.DataFrame(spreds1,columns=['y_pred'])
baocunspreds.to_csv('随机森林预测结果.csv',index=False)

In [ ]:
a=pd.read_csv('随机森林预测结果.csv')
a

In [ ]:
a=np.array(a)
a

In [ ]:
# 随机森林拟合图1
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('随机森林模型',fontsize=25)
# x = range(0,530,1)
plt.plot(label[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(a[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\随机森林预测图\论文随机森林拟合图530.jpg")

In [ ]:
# 随机森林拟合图2
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('随机森林模型',fontsize=25)
x = range(530,1060,1)
plt.plot(x,label[530:1060], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,a[530:1060], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\随机森林预测图\论文随机森林拟合图1060.jpg")

In [ ]:
# 随机森林拟合图3
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('随机森林模型',fontsize=25)
x = range(1060,1590,1)
plt.plot(x,label[1060:1590], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,a[1060:1590], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\随机森林预测图\论文随机森林拟合图1590.jpg")

In [ ]:
# 随机森林拟合图4
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('随机森林模型',fontsize=25)
x = range(1590,2120,1)
plt.plot(x,label[1590:2120], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,a[1590:2120], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\随机森林预测图\论文随机森林拟合图2120.jpg")

In [ ]:
# 随机森林拟合图5
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('随机森林模型',fontsize=25)
x = range(2120,2650,1)
plt.plot(x,label[2120:2650], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,a[2120:2650], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\随机森林预测图\论文随机森林拟合图2650.jpg")

In [ ]:
# 随机森林拟合图6
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('随机森林模型',fontsize=25)
x = range(2650,3180,1)
plt.plot(x,label[2650:3180], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,a[2650:3180], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper right',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\随机森林预测图\论文随机森林拟合图3180.jpg")

Adaboost算法

In [252]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
# 也可以对选择的弱回归器进行参数选择，方式为base_estimator__若回归器参数名
Ada_model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=11, min_samples_split=35, min_samples_leaf=35),
                            n_estimators=50,learning_rate=0.1,loss='linear')
Ada_model.fit(dujiqix_train,dujiqiy_train)
aday_pres=Ada_model.predict(dujiqix_test)
aday_modify = aday_pres*(aday_pres>=0)
adar=r2_score(dujiqiy_test,aday_modify)
adar

In [ ]:
# 也可以对选择的弱回归器进行参数选择，方式为base_estimator__若回归器参数名
Ada_model1=AdaBoostRegressor(DecisionTreeRegressor(),
                            n_estimators=50,learning_rate=0.1,loss='square')
Ada_model1.fit(dujiqix_train,dujiqiy_train)
aday_pres1=Ada_model.predict(dujiqix_test)
aday_modify1 = aday_pres1*(aday_pres1>=0)
adar1=r2_score(dujiqiy_test,aday_modify1)
adar1

In [256]:
apreds = np.array(aday_modify)  # 转换为numpy形式
# labels1 = np.array(dujiqiy_test)
apreds= apreds.reshape((-1,1))
afuture_len= apreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(afuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(afuture_len):
    apreds[:, k] = apreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,apreds))

In [ ]:
r2_score(label,apreds)

In [ ]:
mean_absolute_error(label,apreds)

保存预测结果

In [260]:
apreds1=apreds.reshape(-1,1)
baocunapreds=pd.DataFrame(apreds1,columns=['y_pred'])
baocunapreds.to_csv('AdaBooost预测结果.csv',index=False)

In [ ]:
b=pd.read_csv('AdaBooost预测结果.csv')
b

In [262]:
# # LSTM拟合图
# plt.figure(figsize=(25, 8))
# ax = plt.subplot(111)
# plt.rcParams['font.sans-serif']=['FangSong']
# plt.rcParams['axes.unicode_minus']=False
# plt.grid(color="k", linestyle=":")
# ax.set_title('AdaBoost模型',fontsize=25)
# ax.plot(labels1, "r", label="real", alpha=0.7,ms=7)
# ax.plot(b, "g--", label="pred", alpha=0.8,ms=7)
# plt.xlabel('time',fontsize=20)
# plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
# plt.xticks(fontsize=20)
# plt.yticks(fontsize=20)
# plt.rcParams.update({'font.size': 20})
# plt.legend(fontsize=20)
# plt.savefig("论文AdaBoost拟合图.jpg")

In [267]:
b=np.array(b)

In [ ]:
# 随机森林拟合图1
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
# x = range(0,530,1)
plt.plot(label[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(b[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\AdaBoost预测图\论文AdaBoost拟合图530.jpg")

In [ ]:
# 拟合图2
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
x = range(530,1060,1)
plt.plot(x,label[530:1060], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,b[530:1060], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\AdaBoost预测图\论文AdaBoost拟合图1060.jpg")

In [ ]:
# 拟合图3
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
x = range(1060,1590,1)
plt.plot(x,label[1060:1590], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,b[1060:1590], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\AdaBoost预测图\论文AdaBoost拟合图1590.jpg")

In [ ]:
# 拟合图4
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
x = range(1590,2120,1)
plt.plot(x,label[1590:2120], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,b[1590:2120], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\AdaBoost预测图\论文AdaBoost拟合图2120.jpg")

In [ ]:
# 拟合图5
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
x = range(2120,2650,1)
plt.plot(x,label[2120:2650], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,b[2120:2650], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\AdaBoost预测图\论文AdaBoost拟合图2650.jpg")

In [ ]:
# 拟合图6
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
x = range(2650,3180,1)
plt.plot(x,label[2650:3180], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,b[2650:3180], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper right',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\AdaBoost预测图\论文AdaBoost拟合图3180.jpg")

GBRT

In [289]:
from sklearn.ensemble import GradientBoostingRegressor

In [290]:
# Gbrt_model= GradientBoostingRegressor(n_estimators=50,max_depth=7,min_samples_split=10,learning_rate=0.1,min_samples_leaf=5)#这里使用50个决策树
# Gbrt_model.fit(dujiqix_train,dujiqiy_train)
# gbrty_pres=Gbrt_model.predict(dujiqix_test)
# gbrty_modify= gbrty_pres*(gbrty_pres>=0)
# gbrtr=r2_score(dujiqiy_test,gbrty_modify)
# gbrtr

In [ ]:
Gbrt_model= GradientBoostingRegressor()#这里使用50个决策树
Gbrt_model.fit(dujiqix_train,dujiqiy_train)
gbrty_pres=Gbrt_model.predict(dujiqix_test)
gbrty_modify = gbrty_pres*(gbrty_pres>=0)
gbrtr=r2_score(dujiqiy_test,gbrty_modify)
gbrtr

In [292]:
gpreds = np.array(gbrty_modify)  # 转换为numpy形式
# labels2 = np.array(dujiqiy_test)
gpreds= gpreds.reshape((-1,1))
gfuture_len= gpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(gfuture_len):
#     labels2[:, k] = labels2[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(gfuture_len):
    gpreds[:, k] = gpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
gpreds.shape[1] 

In [ ]:
np.sqrt(mean_squared_error(label,gpreds))

In [ ]:
mean_absolute_error(label,gpreds)

In [ ]:
np.sum(np.absolute(label - gpreds)) / len(dujiqiy_test)

In [ ]:
label

保存预测结果

In [298]:
gpreds=gpreds.reshape(-1,1)
baocungpreds=pd.DataFrame(gpreds,columns=['y_pred'])
baocungpreds.to_csv('GBRT预测结果.csv',index=False)

In [ ]:
c=pd.read_csv('GBRT预测结果.csv')
c

In [25]:
# # LSTM拟合图
# plt.figure(figsize=(25, 8))
# ax = plt.subplot(111)
# plt.rcParams['font.sans-serif']=['FangSong']
# plt.rcParams['axes.unicode_minus']=False
# plt.grid(color="k", linestyle=":")
# ax.set_title('AdaBoost模型',fontsize=25)
# ax.plot(labels, "r", label="real", alpha=0.7,ms=7)
# ax.plot(c, "g--", label="pred", alpha=0.8,ms=7)
# plt.xlabel('time',fontsize=20)
# plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
# plt.xticks(fontsize=20)
# plt.yticks(fontsize=20)
# plt.rcParams.update({'font.size': 20})
# plt.legend(fontsize=20)
# plt.savefig("论文GBRT拟合图.jpg")

In [26]:
c=np.array(c)

In [ ]:
# 拟合图1
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('GBRT模型',fontsize=25)
# x = range(0,530,1)
plt.plot(label[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(c[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\GBRT预测图\论文GBRT拟合图530.jpg")

In [ ]:
# 拟合图2
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
# plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('GBRT模型',fontsize=25)
x = range(530,1060,1)
plt.plot(x,label[530:1060], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,c[530:1060], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=30)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=30)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\GBRT预测图\论文GBRT拟合图1060.jpg")

In [ ]:
# 拟合图3
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('GBRT模型',fontsize=25)
x = range(1060,1590,1)
plt.plot(x,label[1060:1590], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,c[1060:1590], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\GBRT预测图\论文GBRT拟合图1590.jpg")

In [ ]:
# 拟合图4
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('GBRT模型',fontsize=25)
x = range(1590,2120,1)
plt.plot(x,label[1590:2120], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,c[1590:2120], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\GBRT预测图\论文GBRT拟合图2120.jpg")

In [ ]:
# 拟合图5
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('GBRT模型',fontsize=25)
x = range(2120,2650,1)
plt.plot(x,label[2120:2650], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,c[2120:2650], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\GBRT预测图\论文GBRT拟合图2650.jpg")

In [ ]:
# 拟合图6
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('GBRT模型',fontsize=25)
x = range(2650,3180,1)
plt.plot(x,label[2650:3180], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,c[2650:3180], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper right',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\GBRT预测图\论文GBRT拟合图3180.jpg")

xgboost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
other_params = {'learning_rate': 0.1, 'n_estimators': 75, 'max_depth': 4, 'min_child_weight': 5, 
                     'colsample_bytree': 0.7, 'gamma': 0.01,}
xgb_model1=XGBRegressor(**other_params)
xgb_model1.fit(dujiqix_train,dujiqiy_train)
xgby1_pred=xgb_model1.predict(dujiqix_test)
xgby1_modify = xgby1_pred*(xgby1_pred>=0)
xgbr1=r2_score(dujiqiy_test,xgby1_modify)
xgbr1

In [ ]:
other_params1 = {'learning_rate': 0.1, 'n_estimators': 70, 'max_depth': 4, 'min_child_weight': 5, 
                     'colsample_bytree': 0.7, 'gamma': 0.01,
                 }
xgb_model=XGBRegressor(**other_params1)
xgb_model.fit(dujiqix_train,dujiqiy_train)
xgby_pred=xgb_model.predict(dujiqix_test)
xgby_modify = xgby_pred*(xgby_pred>=0)
xgbr=r2_score(dujiqiy_test,xgby_modify)
xgbr

In [54]:
xgbpreds = np.array(xgby1_modify)  # 转换为numpy形式
# labels3 = np.array(dujiqiy_test)
xgbpreds= xgbpreds.reshape((-1,1))
xfuture_len= xgbpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(xfuture_len):
#     labels3[:, k] = labels3[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for j in range(xfuture_len):
    xgbpreds[:, j] = xgbpreds[:, j] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.array(xgby1_modify).reshape(-1,1)

In [ ]:
xgbpreds

In [ ]:
label

In [ ]:
np.sqrt(mean_squared_error(label,xgbpreds))

In [ ]:
mean_absolute_error(label,xgbpreds)

保存预测结果

In [60]:
xgbpreds=xgbpreds.reshape(-1,1)
baocunxgbpreds=pd.DataFrame(xgbpreds,columns=['y_pred'])
baocunxgbpreds.to_csv('XGBoost预测结果.csv',index=False)

In [ ]:
d=pd.read_csv('XGBoost预测结果.csv')
d

In [62]:
d=np.array(d)

In [ ]:
# 拟合图1
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
# x = range(2650,3180,1)
plt.plot(label[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(d[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\XGBoost预测图\论文XGBoost拟合图530.jpg")

In [ ]:
# 拟合图2
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
x = range(530,1060,1)
plt.plot(x,label[530:1060], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,d[530:1060], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\XGBoost预测图\论文XGBoost拟合图1060.jpg")

In [ ]:
# 拟合图3
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
x = range(1060,1590,1)
plt.plot(x,label[1060:1590], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,d[1060:1590], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\XGBoost预测图\论文XGBoost拟合图1590.jpg")

In [ ]:
# 拟合图4
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
x = range(1590,2120,1)
plt.plot(x,label[1590:2120], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,d[1590:2120], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\XGBoost预测图\论文XGBoost拟合图2120.jpg")

In [ ]:
# 拟合图5
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
x = range(2120,2650,1)
plt.plot(x,label[2120:2650], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,d[2120:2650], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\XGBoost预测图\论文XGBoost拟合图2650.jpg")

In [ ]:
# 拟合图6
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
x = range(2650,3180,1)
plt.plot(x,label[2650:3180], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,d[2650:3180], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\XGBoost预测图\论文XGBoost拟合图3180.jpg")

lightGBM

In [69]:
from lightgbm import LGBMRegressor
import lightgbm as lgb

In [ ]:
tmodel_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=36,
                              learning_rate=0.01, n_estimators=430, max_depth=8, 
                              metric='rmse', min_child_samples=4,
                            )#reg_alpha=0.5,reg_lambda=0.5,feature_fraction=0.6,bagging_fraction=0.4,
tmodel_lgb.fit(dujiqix_train,dujiqiy_train)
lgby=tmodel_lgb.predict(dujiqix_test)
lgby_modify=lgby*(lgby>=0)
lgbr=r2_score(dujiqiy_test,lgby_modify)
lgbr

In [ ]:
tmodel_lgb1 = lgb.LGBMRegressor(objective='regression',num_leaves=30,
                              learning_rate=0.01, n_estimators=500, max_depth=8, 
                              metric='rmse', min_child_samples=5,
                            )#reg_alpha=0.5,reg_lambda=0.5,feature_fraction=0.6,bagging_fraction=0.4,
tmodel_lgb1.fit(dujiqix_train,dujiqiy_train)
lgby1=tmodel_lgb1.predict(dujiqix_test)
lgby_modify1=lgby1*(lgby1>=0)
lgbr1=r2_score(dujiqiy_test,lgby_modify1)
lgbr1

In [72]:
lpreds1 = np.array(lgby_modify1)  # 转换为numpy形式
# llabel = np.array(dujiqiy_test)
lpreds1= lpreds1.reshape((-1,1))
lfuture_len= lpreds1.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(lfuture_len):
#     llabel[:, k] = llabel[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(lfuture_len):
    lpreds1[:, k] = lpreds1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,lpreds1))

In [ ]:
mean_absolute_error(label,lpreds1)

In [ ]:
mean_squared_error(llabel,lpreds)

保存预测结果

In [75]:
lpreds1=lpreds1.reshape(-1,1)
baocunlpreds=pd.DataFrame(lpreds1,columns=['y_pred'])
baocunlpreds.to_csv('LightGBM预测结果.csv',index=False)

In [ ]:
e=pd.read_csv('LightGBM预测结果.csv')
e

In [77]:
e=np.array(e)

In [ ]:
# 拟合图1
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('LightGBM模型',fontsize=25)
plt.plot(label[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(e[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\LightGBM预测图\论文LightGBM拟合图530.jpg")

In [ ]:
# 拟合图2
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('LightGBM模型',fontsize=25)
x = range(530,1060,1)
plt.plot(x,label[530:1060], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,e[530:1060], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\LightGBM预测图\论文LightGBM拟合图1060.jpg")

In [ ]:
# 拟合图3
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('LightGBM模型',fontsize=25)
x = range(1060,1590,1)
plt.plot(x,label[1060:1590], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,e[1060:1590], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\LightGBM预测图\论文LightGBM拟合图1590.jpg")

In [ ]:
# 拟合图4
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('LightGBM模型',fontsize=25)
x = range(1590,2120,1)
plt.plot(x,label[1590:2120], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,e[1590:2120], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\LightGBM预测图\论文LightGBM拟合图2120.jpg")

In [ ]:
# 拟合图5
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('LightGBM模型',fontsize=25)
x = range(2120,2650,1)
plt.plot(x,label[2120:2650], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,e[2120:2650], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\LightGBM预测图\论文LightGBM拟合图2650.jpg")

In [ ]:
# 拟合图6
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('LightGBM模型',fontsize=25)
x = range(2650,3180,1)
plt.plot(x,label[2650:3180], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,e[2650:3180], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\LightGBM预测图\论文LightGBM拟合图3180.jpg")

SVR

In [86]:
from sklearn.svm import SVR

In [ ]:
svr_rbf = SVR(kernel='linear', C=10, gamma=1)
svr_rbf.fit(dujiqix_train,dujiqiy_train)
svry_pred= svr_rbf.predict(dujiqix_test)
svry_modify= svry_pred*(svry_pred>=0)
svrr=r2_score(dujiqiy_test,svry_modify)
svrr

In [ ]:
svr_rbf2 = SVR()
svr_rbf2.fit(dujiqix_train,dujiqiy_train)
svry_pred2= svr_rbf2.predict(dujiqix_test)
svry_modify2 = svry_pred2*(svry_pred2>=0)
svrr2=r2_score(dujiqiy_test,svry_modify2)
svrr2

In [90]:
svrpreds = np.array(svry_modify)  # 转换为numpy形式
# labels1 = np.array(y_test)
svrpreds= svrpreds.reshape((-1,1))
svrfuture_len= svrpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(sfuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(svrfuture_len):
    svrpreds[:, k] = svrpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,svrpreds))

In [ ]:
mean_absolute_error(label,svrpreds)

保存预测结果

In [93]:
svrpreds=svrpreds.reshape(-1,1)
baocunsvrpreds=pd.DataFrame(svrpreds,columns=['y_pred'])
baocunsvrpreds.to_csv('SVR预测结果.csv',index=False)

In [ ]:
f=pd.read_csv('SVR预测结果.csv')
f

In [95]:
f=np.array(f)

In [ ]:
# 拟合图1
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('SVR模型',fontsize=25)
plt.plot(label[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(f[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\SVR预测图\论文SVR拟合图530.jpg")

In [ ]:
# 拟合图2
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('SVR模型',fontsize=25)
x = range(530,1060,1)
plt.plot(x,label[530:1060], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,f[530:1060], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\SVR预测图\论文SVR拟合图1060.jpg")

In [ ]:
# 拟合图3
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('SVR模型',fontsize=25)
x = range(1060,1590,1)
plt.plot(x,label[1060:1590], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,f[1060:1590], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\SVR预测图\论文SVR拟合图1590.jpg")

In [ ]:
# 拟合图4
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('SVR模型',fontsize=25)
x = range(1590,2120,1)
plt.plot(x,label[1590:2120], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,f[1590:2120], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\SVR预测图\论文SVR拟合图2120.jpg")

In [ ]:
# 拟合图5
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('SVR模型',fontsize=25)
x = range(2120,2650,1)
plt.plot(x,label[2120:2650], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,f[2120:2650], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\SVR预测图\论文SVR拟合图2650.jpg")

In [ ]:
# 拟合图6
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('SVR模型',fontsize=25)
x = range(2650,3180,1)
plt.plot(x,label[2650:3180], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,f[2650:3180], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\SVR预测图\论文SVR拟合图3180.jpg")

lasso回归

In [102]:
from sklearn.linear_model import LassoCV

In [ ]:
La=LassoCV()
La.fit(dujiqix_train,dujiqiy_train)
lay_predict=La.predict(dujiqix_test)
lay_modify = lay_predict*(lay_predict>=0)
rla=r2_score(dujiqiy_test,lay_modify)
rla

In [104]:
lapreds = np.array(lay_modify)  # 转换为numpy形式
# labels1 = np.array(y_test)
lapreds= lapreds.reshape((-1,1))
lafuture_len= lapreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(lafuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(lafuture_len):
    lapreds[:, k] = lapreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,lapreds))

In [ ]:
mean_absolute_error(label,lapreds)

保存预测结果

In [107]:
lapreds=lapreds.reshape(-1,1)
baocunlapreds=pd.DataFrame(lapreds,columns=['y_pred'])
baocunlapreds.to_csv('Lasso预测结果.csv',index=False)

In [ ]:
g=pd.read_csv('Lasso预测结果.csv')
g

In [109]:
g=np.array(g)

In [ ]:
# 拟合图6
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Lasso模型',fontsize=25)
plt.plot(label[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(g[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Lasso预测图\论文Lasso拟合图530.jpg")

In [ ]:
# 拟合图2
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Lasso模型',fontsize=25)
x = range(530,1060,1)
plt.plot(x,label[530:1060], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,g[530:1060], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Lasso预测图\论文Lasso拟合图1060.jpg")

In [ ]:
# 拟合图3
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Lasso模型',fontsize=25)
x = range(1060,1590,1)
plt.plot(x,label[1060:1590], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,g[1060:1590], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Lasso预测图\论文Lasso拟合图1590.jpg")

In [ ]:
# 拟合图4
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Lasso模型',fontsize=25)
x = range(1590,2120,1)
plt.plot(x,label[1590:2120], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,g[1590:2120], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Lasso预测图\论文Lasso拟合图2120.jpg")

In [ ]:
# 拟合图5
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Lasso模型',fontsize=25)
x = range(2120,2650,1)
plt.plot(x,label[2120:2650], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,g[2120:2650], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Lasso预测图\论文Lasso拟合图2650.jpg")

In [ ]:
# 拟合图6
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Lasso模型',fontsize=25)
x = range(2650,3180,1)
plt.plot(x,label[2650:3180], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,g[2650:3180], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Lasso预测图\论文Lasso拟合图3180.jpg")

# 深度学习算法

In [65]:
# 读取训练集
dushendux_train=pd.read_csv('shendux_train.csv')
dushendux_train1=np.array(dushendux_train)
dushendux_train2=np.reshape(dushendux_train1,(-1,120,8))

dushenduy_train=pd.read_csv('shenduy_train.csv')
dushenduy_train2=np.array(dushenduy_train)
# 读取测试集
dushendux_test=pd.read_csv('shendux_test.csv')
dushendux_test1=np.array(dushendux_test)
dushendux_test2=np.reshape(dushendux_test1,(-1,120,8))

dushenduy_test=pd.read_csv('shenduy_test.csv')
dushenduy_test2=np.array(dushenduy_test)

In [ ]:
dushendux_train2

In [ ]:
dushendux_train2.shape

In [68]:
# # # 构造批数据
def create_batch_dataset(x,y,train=True,buffer_size=1000,batch_size=128):#buffer_size=1000表示可以打乱窗口里面的数据
    batch_data=tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y)))#数据封装，tensor类型
    if train:
        return batch_data.cache().shuffle(buffer_size).batch(batch_size)
    else:
        return batch_data.batch(batch_size)

In [69]:
train_data_single=create_batch_dataset(dushendux_train2, dushenduy_train2,train=True)
val_data_single=create_batch_dataset(dushendux_test2,dushenduy_test2,train=False)

In [ ]:
train_data_single

In [ ]:
dushendux_train2.shape[-2:]

In [ ]:
dushendux_train2.shape

In [40]:
# 添加注意力机制
def attention_bilstm(inputs):
    """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
    hidden_states = inputs
    hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
    attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
    pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
    attention_vector = Dense(128, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return attention_vector

In [41]:
# 添加注意力机制
def attention_bilstm64(inputs):
    """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
    hidden_states = inputs
    hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
    attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
    pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
    attention_vector = Dense(64, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return attention_vector

Bi-LSTM

In [ ]:
single_step_model1 = tf.keras.models.Sequential()
single_step_model1.add(tf.keras.layers.Bidirectional(LSTM(32,input_shape=dushendux_train2.shape[-2:],return_sequences=True)))
single_step_model1.add(Dropout(0.2))
single_step_model1.add(tf.keras.layers.Bidirectional(LSTM(64,input_shape=dushendux_train2.shape[-2:])))
single_step_model1.add(Dropout(0.2))
single_step_model1.add(tf.keras.layers.Dense(1))
single_step_model1.add(tf.keras.layers.Activation('linear'))
single_step_model1.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')

single_step_history1 = single_step_model1.fit(train_data_single, epochs=500,
#                                             steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            )#validation_steps=50

In [ ]:
# 预测
bilstm_pres=single_step_model1.predict(dushendux_test2,verbose=1)

In [ ]:
bilstmy_modify1 = bilstm_pres*(bilstm_pres>=0)
bilstmr=r2_score(dushenduy_test2,bilstmy_modify1)
bilstmr

保存模型

In [ ]:
single_step_model1.save('论文savebiLSTM_model')

In [35]:
# single_step_model1.save('论文savebiLSTM_model')
pd.DataFrame(single_step_history1.history).to_csv('savebiLSTM_history.csv', index=False)
# bilstm_data=pd.DataFrame(bilstmy_modify1,columns=['bi-lstm_pred'])
# bilstm_data.to_csv('savebiLSTM_pred.csv',index=False)

In [39]:
bilstm_data=pd.DataFrame(bilstmy_modify1,columns=['bi-lstm_pred'])
bilstm_data.to_csv('savebiLSTM_pred.csv',index=False)

In [36]:
dubilstm_history=pd.read_csv('savebiLSTM_history.csv')

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
plt.plot(dubilstm_history['val_loss'],label=['Bi-LSTM_loss_value'])
plt.xlabel('number',fontsize=25)
plt.ylabel('Bi-LSTM_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
# plt.rcParams.update({'font.size': 14})
# plt.ylim(0,None)
# plt.xlim(0,None)
plt.legend(fontsize=20)
# plt.legend(loc='best')
plt.savefig("论文bilstm损失图.jpg")
plt.show()

In [ ]:
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)

In [77]:
labels = np.array(dushenduy_test2)
future_len = labels.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(future_len):
    labels[:, k] = labels[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
labels

In [49]:
bi=pd.read_csv('savebiLSTM_pred.csv')

In [50]:
 # 数据逆归一化
preds = np.array(bi)  # 转换为numpy形式
preds= preds.reshape((-1,1))
for k in range(future_len):
    preds[:, k] = preds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [89]:
from matplotlib.pyplot import MultipleLocator

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM模型',fontsize=25)
ax.plot(labels[:500], "r", label="real", alpha=0.7,ms=8)
ax.plot(preds[:500], "g-.", label="pred", alpha=0.7,ms=8)
plt.xlabel('time/h',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
# plt.tight_layout()
plt.savefig("论文Bi-LSTM拟合图500.jpg")

Bi-LSTM第2次训练

In [124]:
from keras.callbacks import ModelCheckpoint
filepath2 = "E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Bi-LSTM最好模型第二版本\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint2 = ModelCheckpoint(filepath=filepath2,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
single_step_model2 = tf.keras.models.Sequential()
single_step_model2.add(tf.keras.layers.Bidirectional(LSTM(128,input_shape=dushendux_train2.shape[-2:],return_sequences=True)))
single_step_model2.add(Dropout(0.2))
single_step_model2.add(tf.keras.layers.Bidirectional(LSTM(64,input_shape=dushendux_train2.shape[-2:])))
single_step_model2.add(tf.keras.layers.Dense(1))
# single_step_model1.add(tf.keras.layers.Activation('linear'))
single_step_model2.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')
single_step_history2 = single_step_model2.fit(train_data_single, epochs=800, validation_data=val_data_single,callbacks=checkpoint2)

In [142]:
from keras.models import load_model

In [143]:
bilstm_model1=load_model("E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Bi-LSTM最好模型第二版本\weights-improvement-564-0.00.hdf5")

In [ ]:
zui2=bilstm_model1.predict(dushendux_test2,verbose=1)

In [ ]:
zui2_modify = zui2*(zui2>=0)
bilstmr2=r2_score(dushenduy_test2,zui2_modify)
bilstmr2

In [ ]:
bilstm_model1.evaluate(dushendux_test2,dushenduy_test2)

第三版本

In [123]:
from keras.callbacks import ModelCheckpoint
filepath3 = "Bi-LSTM最好模型第三版本\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint3 = ModelCheckpoint(filepath=filepath3,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
single_step_model3 = tf.keras.models.Sequential()
single_step_model3.add(tf.keras.layers.Bidirectional(LSTM(128,input_shape=dushendux_train2.shape[-2:],return_sequences=True)))
single_step_model3.add(Dropout(0.5))
single_step_model3.add(tf.keras.layers.Bidirectional(LSTM(64,input_shape=dushendux_train2.shape[-2:])))
single_step_model3.add(Dropout(0.2))
single_step_model3.add(tf.keras.layers.Dense(1))
# single_step_model1.add(tf.keras.layers.Activation('linear'))
single_step_model3.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='mse',metrics='mae')
single_step_history3= single_step_model3.fit(train_data_single, epochs=800, validation_data=val_data_single,callbacks=checkpoint3)

In [124]:
bilstm_model3=load_model("Bi-LSTM最好模型第三版本\weights-improvement-496-0.00.hdf5")

In [ ]:
zui3=bilstm_model3.predict(dushendux_test2,verbose=1)

In [ ]:
zui3_modify = zui3*(zui3>=0)
bilstmr3=r2_score(dushenduy_test2,zui3_modify)
bilstmr3

In [127]:
pd.DataFrame(single_step_history3.history).to_csv('savebiLSTM_history3.csv', index=False)

In [ ]:
single_step_model3.save('论文3savebiLSTM_model')

第4版本

In [42]:
from keras.callbacks import ModelCheckpoint
filepath4 = "Bi-LSTM第四版本\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint4 = ModelCheckpoint(filepath=filepath4,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
bilstm1=Bidirectional(LSTM(128,dropout=0.3,return_sequences=True))(model_input)
bilstm2=Bidirectional(LSTM(64,dropout=0.3))(bilstm1)
den1=Dense(1)(bilstm2)
bilstm_model4=Model(model_input,den1)
adaa=optimizers.Adam(lr=0.0005)
bilstm_model4.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
bilstm_model_history4 = bilstm_model4.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=checkpoint4)

第一版本

In [130]:
filepathnew =  "E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Bi-LSTM-Attention最好模型第一版本\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpointnew = ModelCheckpoint(filepath=filepathnew,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
con1=Conv1D(64,3,activation='linear',input_shape=(dushendux_train2.shape[-2],dushendux_train2.shape[-1]))(model_input)
# max1=MaxPooling2D((2,2)))(con1)
max1 = MaxPooling1D(pool_size = 2, name = 'max_pool_1')(con1)
# flat1=Flatten()(max1)
bi_lstm_att1=Bidirectional(LSTM(128,dropout=0.5,return_sequences=True))(max1)
# t1=Dropout(0.5)(bi_lstm_att1)
bi_gru2=Bidirectional(LSTM(64,dropout=0.3,return_sequences=True))(bi_lstm_att1)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
# t11=Dropout(0.2)(bi_gru2)
attbigru1=attention_bilstm(bi_gru2)
# a=Dropout(0.2)(attbigru)
den1=Dense(1)(attbigru1)
newattbilstm_model1=Model(model_input,den1)
newattbilstm_model1.compile(optimizer='Adam', loss='mse',metrics='mae')# metrics=['accuracy']
newattbilstm_model_history = newattbilstm_model1.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpointnew)

In [132]:
bilstm_model1=load_model( "E:\jupyter\论文测试版本\huangwenchao\北京昌平PM2.5预测\Bi-LSTM-Attention最好模型第一版本\weights-improvement-479-0.00.hdf5")

In [ ]:
zui1=bilstm_model1.predict(dushendux_test2,verbose=1)

In [ ]:
zui1

In [ ]:
zui1_modify = zui1*(zui1>=0)
bilstmr1=r2_score(dushenduy_test2,zui1_modify)
bilstmr1

# 读取最好版本937

In [35]:
# 读取最好模型
attbilstm_model1=load_model("bilstmattention最好模型第二十五版\weights-improvement-1422-0.00.hdf5")

In [ ]:
zui937=attbilstm_model1.predict(dushendux_test2,verbose=1)

In [ ]:
zui937_modify = zui937*(zui937>=0)
attbilstmr1=r2_score(dushenduy_test2,zui937_modify)
attbilstmr1

In [40]:
 # 数据逆归一化
attbilstmpreds = np.array(zui937_modify)  # 转换为numpy形式
attbilstmpreds= attbilstmpreds.reshape((-1,1))
for k in range(future_len):
    attbilstmpreds[:, k] = attbilstmpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
attbilstmpreds

In [43]:
# 保存预测结果
attbilstm_data=pd.DataFrame(attbilstmpreds,columns=['att-bi-lstm_pred'])
attbilstm_data.to_csv('论文注意力机制937预测结果.csv',index=False)

In [ ]:
# 读取保存数据
duquatt=pd.read_csv('论文注意力机制937预测结果.csv')
r2_score(labels,duquatt)

In [48]:
duquatt=np.array(duquatt)

In [ ]:
# 画图
# 拟合图1
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=25)
plt.plot(labels[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(duquatt[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文预测处理2\北京昌平PM2.5预测\注意力机制预测图\论文注意力机制拟合图530.jpg")

In [ ]:
# 拟合图2
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=25)
x = range(530,1060,1)
plt.plot(x,labels[530:1060], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,duquatt[530:1060], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文预测处理2\北京昌平PM2.5预测\注意力机制预测图\论文注意力机制拟合图1060.jpg")

In [ ]:
# 拟合图3
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=25)
x = range(1060,1590,1)
plt.plot(x,labels[1060:1590], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,duquatt[1060:1590], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文预测处理2\北京昌平PM2.5预测\注意力机制预测图\论文注意力机制拟合图1590.jpg")

In [ ]:
# 拟合图4
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=25)
x = range(1590,2120,1)
plt.plot(x,labels[1590:2120], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,duquatt[1590:2120], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文预测处理2\北京昌平PM2.5预测\注意力机制预测图\论文注意力机制拟合图2120.jpg")

In [ ]:
# 拟合图5
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=25)
x = range(2120,2650,1)
plt.plot(x,labels[2120:2650], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,duquatt[2120:2650], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文预测处理2\北京昌平PM2.5预测\注意力机制预测图\论文注意力机制拟合图2650.jpg")

In [ ]:
# 拟合图6
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=25)
x = range(2650,3180,1)
plt.plot(x,labels[2650:3180], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,duquatt[2650:3180], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.tick_params(labelsize=20) #刻度字体大小20
plt.legend(loc='upper left',fontsize=20)
# plt.tight_layout()
plt.savefig("E:\jupyter\论文预测处理2\北京昌平PM2.5预测\注意力机制预测图\论文注意力机制拟合图3180.jpg")

# 读最终版本LSTM-Attention

In [73]:
# 读取最好模型
lstmatt_model6=load_model("lstmattention第6版\weights-improvement-1044-0.00.hdf5")

In [ ]:
zui9453=lstmatt_model6.predict(dushendux_test2,verbose=1)

In [ ]:
zui9453_modify = zui9453*(zui9453>=0)
attlstmr6=r2_score(dushenduy_test2,zui9453_modify)
attlstmr6

In [79]:
 # 数据逆归一化
attlstmpreds = np.array(zui9453_modify)  # 转换为numpy形式
attlstmpreds= attlstmpreds.reshape((-1,1))
for k in range(future_len):
    attlstmpreds[:, k] = attlstmpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [80]:
# 保存预测结果
attlstm_data=pd.DataFrame(attlstmpreds,columns=['att-lstm_pred'])
attlstm_data.to_csv('论文注意力机制946预测结果.csv',index=False)

In [ ]:
# 读取保存数据
duquattlstm=pd.read_csv('论文注意力机制946预测结果.csv')
r2_score(labels,duquattlstm)

读取模型。绘制损失图

In [87]:
# 读取最好模型
duhistory_lstmatt=pd.read_csv("论文0.9453saveattLSTM_history.csv")

In [ ]:
duhistory_lstmatt

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
plt.plot(duhistory_lstmatt['val_loss'],label=['LSTM-Attention_loss_value'])
plt.xlabel('time/h',fontsize=25)
plt.ylabel('LSTM-Attention_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
# plt.rcParams.update({'font.size': 14})
# plt.ylim(0,None)
# plt.xlim(0,None)
plt.legend(fontsize=20)
# plt.legend(loc='best')
plt.savefig("论文LSTM-Attention损失图.jpg")
plt.show()

# LSTM模型

In [30]:
lstmfilepath1 =  "E:\jupyter\论文预测处理这是最后的\北京昌平PM2.5预测\LSTM第一版本\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
lstmcheckpoint1 = ModelCheckpoint(filepath=lstmfilepath1,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
lstm1=LSTM(128,dropout=0.2,return_sequences=True)(model_input)
lstm2=LSTM(64,dropout=0.2)(lstm1)
den1=Dense(1)(lstm2)
lstm_model1=Model(model_input,den1)
adaa=optimizers.Adam(lr=0.001)
lstm_model1.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
lstm_model_history = lstm_model1.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=lstmcheckpoint1)

In [ ]:
lstmyuce=lstm_model1.predict(dushendux_test2,verbose=1)

In [ ]:
lstmyuce_modify = lstmyuce*(lstmyuce>=0)
lstmr1=r2_score(dushenduy_test2,lstmyuce_modify)
lstmr1

保存模型

In [ ]:
lstm_model1.save('论文0.939saveLSTM_model')
pd.DataFrame(lstm_model_history.history).to_csv('save0.939LSTM_history.csv', index=False)
lstm_data=pd.DataFrame(lstmyuce_modify,columns=['lstm_pred'])
lstm_data.to_csv('save0.939LSTM_pred.csv',index=False)

In [41]:
cnnbilstmfilepath1 =  "E:\jupyter\论文预测处理这是最后的\北京昌平PM2.5预测\CNNBILSTM第一版本\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
cnnbilstmcheckpoint1 = ModelCheckpoint(filepath=cnnbilstmfilepath1,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
con1=Conv1D(16,1,input_shape=(dushendux_train2.shape[-2],dushendux_train2.shape[-1]))(model_input)
max1 = MaxPooling1D(pool_size = 1, name = 'max_pool_1')(con1)
con2=Conv1D(16,1,input_shape=(dushendux_train2.shape[-2],dushendux_train2.shape[-1]))(max1)
cnn_bilstm_att1=Bidirectional(LSTM(128,dropout=0.2,recurrent_dropout=0.5,return_sequences=True))(con2)
# cnn_bilstm2_att1=Bidirectional(LSTM(64,dropout=0.3,recurrent_dropout=0.5,return_sequences=True))(cnn_bilstm_att1)
attbigru1=attention_bilstm(cnn_bilstm_att1)
den1=Dense(1)(attbigru1)
cnnattbilstm_model1=Model(model_input,den1)
adaa=optimizers.Adam(lr=0.001)
cnnattbilstm_model1.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
cnnattbilstm_model_history = cnnattbilstm_model1.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=cnnbilstmcheckpoint1)

将xgboost与bilstm-attention结合

In [117]:
bilstmfilepath=  "E:\jupyter\论文预测处理这是最后的\北京昌平PM2.5预测\BILSTM第1版本\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
bilstmcheckpoint = ModelCheckpoint(filepath=bilstmfilepath,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(128,dropout=0.1,input_shape=dushendux_train2.shape[-2:],return_sequences=True)))
bilstm_model.add(Bidirectional(LSTM(64,dropout=0.1,input_shape=dushendux_train2.shape[-2:])))
bilstm_model.add(Dense(1))
# single_step_model1.add(tf.keras.layers.Activation('linear'))
bilstm_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0005), loss='mse',metrics='mae')
bilstm_history = bilstm_model.fit(train_data_single, epochs=1500, validation_data=val_data_single,callbacks=bilstmcheckpoint)